In [1]:
!pip install opencv-python
!pip install opencv-contrib-python
!pip install ffmpeg moviepy


import sys
import cv2
import numpy as np
import datetime
import math
import moviepy.editor as mp
from operator import itemgetter
import pandas as pd
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=a2934f1ca6a135628b9f41073574380154b52c1c43e49b633afb960e715f0194
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2433024/45929032 bytes (5.3%)5201920/45929032 bytes (11.3%)7946240/45929032 bytes (17.3%)

In [2]:
def find_shot_transition_start(cap):
    start = time.time()
    threshold = 60

  # average intensity of pixels in a frame
    last_mean = 0 

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = 1
    duration = frame_count / fps

    while True:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
      #  get the image of current frame
        (rv, im) = cap.read()
        # if cant get the frame -- break
        # run through frames in 4 times fps for quick processing
        frame_count = frame_count + 20 * fps
        if not rv:
            break
        # get mean intensivity of pixels (colors in rgb)
        frame_mean = np.sum(im) / float(im.shape[0] * im.shape[1] * im.shape[2])

        if abs(frame_mean - last_mean) >= threshold:
            return cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

        last_mean = frame_mean

    return 0

In [3]:
def find_shot_transition_end(cap):
    start = time.time()
    threshold = 60
  # average intensity of pixels in a frame
    last_mean = 0 

  # video number of frames (for getting last index) and fps length in seconds
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT) - 2
    duration = frame_count / fps

    while True:
        # set the frame to the video
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
        (rv, im) = cap.read()
        # run through frames in 4 times fps for quick processing
        frame_count = frame_count - 20 * fps

        if not rv:
            break

        frame_mean = np.sum(im) / float(im.shape[0] * im.shape[1] * im.shape[2])

        if abs(frame_mean - last_mean) >= threshold:
            return cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

        last_mean = frame_mean

        end = time.time()

        if (end - start) >= 900:
            print("Timeout: 900 seconds: ", end - start)
            return duration
        
    return duration

In [4]:
def get_sec_hours(time_str):
    """Get seconds from time."""
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

def get_sec_minutes(time_str):
    """Get seconds from time."""
    m, s = time_str.split(':')
    return int(m) * 60 + int(s)

def get_seconds(str_time):
    if len(str_time.split(':')) == 2:
        return get_sec_minutes(str_time)
    elif len(str_time.split(':')) == 3:
        return get_sec_hours(str_time)
    else:
        return int(str_time)

In [5]:
import pandas as pd
import datetime
df = pd.read_csv('dataset.csv', sep=';',header=None)
sound_data = pd.read_csv('sound_results.csv', sep=';',header=None)

names = []
begin = []
end = []
correct_start = []
correct_end = []

print('=======================================')
for i in range(len(df)):
    name = df.values[i][0]
    video_name = 'drive/MyDrive/dataset/' + name

    print(name)
    names.append(name)

    print("Correct Answer: ", get_seconds(df.values[i][1]), " - ", get_seconds(df.values[i][2]))
    correct_t1 = get_seconds(df.values[i][1])
    correct_t2 = get_seconds(df.values[i][2])
    correct_start.append(correct_t1)
    correct_end.append(correct_t2)
    cap = cv2.VideoCapture(video_name)
    print("Sound Answer: ", int(sound_data.values[i][2]), " - ", int(sound_data.values[i][3]))
    t1 = find_shot_transition_start(cap)
    t2 = find_shot_transition_end(cap)
    begin.append(t1)
    end.append(t2)
    cap.release()
    print("Shot Transition Answer: ", t1, " - ", t2)
    print('=======================================')

2022-03-01 16_12 Параллельное программирование, 3ВШЭ, весна 2022.mp4
Correct Answer:  900  -  9844
Sound Answer:  869  -  9060
Shot Transition Answer:  860.0400000000001  -  9873.36
2022-04-19 16_55 ПМИ3 Высокопроизводительные вычисления.mp4
Correct Answer:  2190  -  8410
Sound Answer:  2159  -  5835
Shot Transition Answer:  2200.04  -  8401.36
2022-05-16 19_29 ПМИ3 Высокопроизводительные вычисления.mp4
Correct Answer:  275  -  3885
Sound Answer:  164  -  180
Shot Transition Answer:  280.04  -  3872.6000000000004
2022-03-14 14_31 Software Design, лекция, ВШЭ3, весна 2022.mp4
Correct Answer:  550  -  5395
Sound Answer:  509  -  5385
Shot Transition Answer:  20.040000000000003  -  5445.6
2022-03-21 14_37 Software Design, лекция, ВШЭ3, весна 2022.mp4
Correct Answer:  270  -  5095
Sound Answer:  224  -  5115
Shot Transition Answer:  220.04000000000002  -  5141.400000000001
2022-04-11 14_31 Software Design, лекция, ВШЭ3, весна 2022.mp4
Correct Answer:  845  -  5560
Sound Answer:  839  -  55

In [6]:
result_data = np.array([correct_start, correct_end, begin, end])
result_data_transpose = result_data.transpose()
np.savetxt("shot_transition_results.csv", result_data_transpose, fmt="%d", delimiter=";")